In [3881]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

import medmnist
from medmnist import INFO, Evaluator

In [3882]:
print(f"MedMNIST v{medmnist.__version__} @ {medmnist.HOMEPAGE}")

MedMNIST v2.2.3 @ https://github.com/MedMNIST/MedMNIST/


In [3883]:
# data_flag = 'pathmnist'
data_flag = 'breastmnist'
download = True

NUM_EPOCHS = 50
BATCH_SIZE = 32
lr = 0.001

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

In [3884]:
# preprocessing
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

# load the data
train_dataset = DataClass(split='train', transform=data_transform, download=download)
test_dataset = DataClass(split='test', transform=data_transform, download=download)

pil_dataset = DataClass(split='train', download=download)

Using downloaded and verified file: /home/caesar/.medmnist/breastmnist.npz
Using downloaded and verified file: /home/caesar/.medmnist/breastmnist.npz
Using downloaded and verified file: /home/caesar/.medmnist/breastmnist.npz


In [3885]:
print(train_dataset)

Dataset BreastMNIST (breastmnist)
    Number of datapoints: 546
    Root location: /home/caesar/.medmnist
    Split: train
    Task: binary-class
    Number of channels: 1
    Meaning of labels: {'0': 'malignant', '1': 'normal, benign'}
    Number of samples: {'train': 546, 'val': 78, 'test': 156}
    Description: The BreastMNIST is based on a dataset of 780 breast ultrasound images. It is categorized into 3 classes: normal, benign, and malignant. As we use low-resolution images, we simplify the task into binary classification by combining normal and benign as positive and classifying them against malignant as negative. We split the source dataset with a ratio of 7:1:2 into training, validation and test set. The source images of 1×500×500 are resized into 1×28×28.
    License: CC BY 4.0


In [3886]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

EMBEDDING_LEN = 100

# Load MNIST dataset
(x_train, y_train) = (train_dataset.imgs, train_dataset.labels)
(x_test, y_test) = (test_dataset.imgs, test_dataset.labels)

In [3887]:
print(x_train.shape)
print(y_train.shape)

(546, 28, 28)
(546, 1)


In [3888]:
# Preprocess images
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# Get the user-specified embedding length
embedding_length = EMBEDDING_LEN

# Define a CNN model with variable embedding length
# model = Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
#     MaxPooling2D((2, 2)),
#     Flatten(),
#     Dense(embedding_length, activation='relu'),  # Embedding layer with variable length
# ])  # Remove the final classification layer

model = Sequential([
    # Initial convolutional block
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    # Second convolutional block for deeper feature extraction
    Conv2D(64, (3, 3), activation='relu'),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    # Flatten and dense layers
    Flatten(),
    Dense(128, activation='relu'),
    # Dropout(0.5),  # Regularization to prevent overfitting
    Dense(embedding_length, activation='relu')  # Variable-length embedding layer
])

In [3889]:
# Compile the model (important for generating meaningful embeddings)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [3890]:
# Extract embeddings for all images in the training set
embeddings_train = model.predict(x_train)
print(embeddings_train.shape)  # Output: (60000, embedding_length)

18/18 [==============================] - 0s 7ms/step
(546, 100)


In [3891]:
# Optionally, extract embeddings for test images
embeddings_test = model.predict(x_test)
print(embeddings_test.shape)  # Output: (10000, embedding_length)

5/5 [==============================] - 0s 7ms/step
(156, 100)


In [3892]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
#(, y_train), (x_test, y_test)

(546, 28, 28, 1)
(546, 1)
(156, 28, 28, 1)
(156, 1)


In [3893]:
import numpy as np
from sklearn.svm import SVC

In [3894]:
# Load your dataset (assuming it's in NumPy arrays)
x_train = embeddings_train
y_train = y_train
x_test = embeddings_test
y_test = y_test

In [3895]:
# Create an SVM classifier with a suitable kernel
clf = SVC(kernel='rbf', C=1.0, gamma='scale')  # RBF kernel often works well

In [3896]:
# Fit the classifier on the training data
y_train = y_train.ravel()
clf.fit(x_train, y_train)

SVC()

In [3897]:
# Make predictions on the test data
y_pred = clf.predict(x_test)

In [3898]:
# Evaluate model performance
y_test = y_test.ravel()
accuracy = clf.score(x_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.782051282051282
